In [13]:
import pandas as pd

data = pd.read_csv('county-data.csv')
county_df = pd.DataFrame(data)

data = pd.read_csv('municipal-data.csv')
muni_df = pd.DataFrame(data)


C:\Users\Nicole\AppData\Local\Temp\ipykernel_35612\2676995073.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('municipal-data.csv')


In [14]:
columns_to_drop = ['RANK', 'SUMLEV', 'PLACE', 'COUSUB', 'CONCIT', 'PRIMGEO_FLAG', 'FUNCSTAT', 'ESTIMATESBASE2020', 'POPESTIMATE2020', 'POPESTIMATE2021', 'POPESTIMATE2022', 'Type', 'UID', 'Population Sum', '% of Population', 'Total']
county_df = county_df.drop(columns=columns_to_drop)

print(county_df)

      STATE  COUNTY                NAME      STNAME
0         6      37  Los Angeles County  California
1        17      31         Cook County    Illinois
2        48     201       Harris County       Texas
3         4      13     Maricopa County     Arizona
4         6      73    San Diego County  California
...     ...     ...                 ...         ...
3089     31     117    McPherson County    Nebraska
3090     48     261       Kenedy County       Texas
3091     48     269         King County       Texas
3092     15       5      Kalawao County      Hawaii
3093     48     301       Loving County       Texas

[3094 rows x 4 columns]


In [15]:
columns_to_drop = ['RANK', 'Population Sum', '% of Population']

muni_df = muni_df.drop(columns=columns_to_drop)\
                .rename(columns={'NAME': 'MUNI NAME'})

print(muni_df)

       SUMLEV  STATE  COUNTY  PLACE  COUSUB  CONCIT  PRIMGEO_FLAG FUNCSTAT   
0         162     36       0  51000       0       0             0        A  \
1         162      6       0  44000       0       0             0        A   
2          61     39     123      0    1322       0             0        A   
3         162     48       0  35000       0       0             0        A   
4         162      4       0  55000       0       0             0        A   
...       ...    ...     ...    ...     ...     ...           ...      ...   
36568     162     48       0  50184       0       0             0        A   
36569      61     50       9      0    2162       0             1        N   
36570      61     50       9      0   39775       0             1        I   
36571      61     50       9      0   76337       0             1        N   
36572      61     55      27      0   87225       0             1        A   

               MUNI NAME      STNAME  ... ESTIMATESBASE2020  PO

### Merge with County List

In [16]:
merged_data = pd.merge(muni_df, county_df, how='left', on=['STATE', 'COUNTY'])

merged_data['COUNTY NAME'] = merged_data['NAME']

print(merged_data)


       SUMLEV  STATE  COUNTY  PLACE  COUSUB  CONCIT  PRIMGEO_FLAG FUNCSTAT   
0         162     36       0  51000       0       0             0        A  \
1         162      6       0  44000       0       0             0        A   
2          61     39     123      0    1322       0             0        A   
3         162     48       0  35000       0       0             0        A   
4         162      4       0  55000       0       0             0        A   
...       ...    ...     ...    ...     ...     ...           ...      ...   
41360     162     48       0  50184       0       0             0        A   
41361      61     50       9      0    2162       0             1        N   
41362      61     50       9      0   39775       0             1        I   
41363      61     50       9      0   76337       0             1        N   
41364      61     55      27      0   87225       0             1        A   

               MUNI NAME    STNAME_x  ... POPESTIMATE2022  IS C

In [17]:
columns_to_drop = ['STNAME_y', 'IS COUNTY', 'STATESTATE', 'NAME', 'COUNTY DISTINCTION']

merged_data = merged_data.drop(columns=columns_to_drop)\
                        .rename(columns={'STNAME_x': 'STNAME'})

print(muni_df)

       SUMLEV  STATE  COUNTY  PLACE  COUSUB  CONCIT  PRIMGEO_FLAG FUNCSTAT   
0         162     36       0  51000       0       0             0        A  \
1         162      6       0  44000       0       0             0        A   
2          61     39     123      0    1322       0             0        A   
3         162     48       0  35000       0       0             0        A   
4         162      4       0  55000       0       0             0        A   
...       ...    ...     ...    ...     ...     ...           ...      ...   
36568     162     48       0  50184       0       0             0        A   
36569      61     50       9      0    2162       0             1        N   
36570      61     50       9      0   39775       0             1        I   
36571      61     50       9      0   76337       0             1        N   
36572      61     55      27      0   87225       0             1        A   

               MUNI NAME      STNAME  ... ESTIMATESBASE2020  PO

In [18]:
# government = merged_data[merged_data['TYPE'] == 'government'].index
# merged_data.drop(government, inplace=True)
# region = merged_data[merged_data['TYPE'] == 'Region'].index
# merged_data.drop(region, inplace=True)
# parish = merged_data[merged_data['TYPE'] == 'Parish'].index
# merged_data.drop(parish, inplace=True)
# ut = merged_data[merged_data['TYPE'] == 'UT'].index
# merged_data.drop(ut, inplace=True)
# bow = merged_data[merged_data['TYPE'] == 'Bow'].index
# merged_data.drop(bow, inplace=True)
# cdp = merged_data[merged_data['TYPE'] == 'CDP'].index
# merged_data.drop(cdp, inplace=True)

types_to_drop = ['government', 'Region', 'Parish', 'UT', 'Bow', 'CDP']
merged_data = merged_data[~merged_data['TYPE'].isin(types_to_drop)]


In [19]:
merged_data.drop_duplicates(subset='UID Full', keep=False, inplace=True)

In [20]:
merged_data.to_csv('updated-municipalities.csv', index=False)

### Merge with Census Place FIPS Code List

In [21]:
census_df = pd.read_csv('census-place-list.csv', sep='|')

census_df = census_df.astype(str)

# Replace pipe delimiters with commas
census_df = census_df.applymap(lambda x: x.replace('|', ','))

# Write the modified data back to a CSV file
census_df.to_csv('census-place-reformatted.csv', index=False)
census_df = pd.read_csv('census-place-reformatted.csv')

In [22]:
columns_to_drop = ['STATE', 'PLACENAME', 'TYPE', 'CLASSFP', 'FUNCSTAT', 'PLACENS']

census_df = census_df.drop(columns=columns_to_drop)\
                    .rename(columns={'PLACEFP':'PLACE', 'STATEFP': 'STATE'})

print(census_df)

       STATE  PLACE           COUNTIES
0          1    100    Chambers County
1          1    124       Henry County
2          1    460   Jefferson County
3          1    484     Winston County
4          1    676        Hale County
...      ...    ...                ...
32183     78  28000    St. John Island
32184     78  37900   St. Croix Island
32185     78  39700   St. Croix Island
32186     78  65530  St. Thomas Island
32187     78  78300  St. Thomas Island

[32188 rows x 3 columns]


In [23]:
muni_data2 = pd.read_csv('updated-municipalities.csv')
muni_data2.drop_duplicates(subset='UID Full', keep=False, inplace=True)

merged_data = pd.merge(muni_data2, census_df, how='left', on=['STATE', 'PLACE'])

merged_data.rename(columns={'COUNTIES': 'COUNTY NAME'}, inplace=True)

print(merged_data)

       SUMLEV  STATE  COUNTY  PLACE  COUSUB  CONCIT  PRIMGEO_FLAG FUNCSTAT   
0         162     36       0  51000       0       0             0        A  \
1         162      6       0  44000       0       0             0        A   
2          61     39     123      0    1322       0             0        A   
3         162     48       0  35000       0       0             0        A   
4         162      4       0  55000       0       0             0        A   
...       ...    ...     ...    ...     ...     ...           ...      ...   
34800     162     48       0  50184       0       0             0        A   
34801      61     50       9      0    2162       0             1        N   
34802      61     50       9      0   39775       0             1        I   
34803      61     50       9      0   76337       0             1        N   
34804      61     55      27      0   87225       0             1        A   

               MUNI NAME      STNAME  ESTIMATESBASE2020  POPEST

In [24]:
# merged_data.sort_values('IS MULTIPLE', inplace=True, ascending=False)

merged_data.sort_values('POPESTIMATE2022', inplace=True, ascending=False)

merged_data.to_csv('updated-municipalities2.csv', index=False)